In [ ]:
from google.colab import files
uploades = files.upload()

Saving Total_Data.xlsx to Total_Data.xlsx


In [265]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

Below DataFrame consists all the data entries of Premier League between 2003 - 
2019

In [266]:
df = pd.read_excel('Total_Data.xlsx') 

Functions

In [278]:
# Gives us the dataframe of team a_vs_team b for all past years
def a_vs_b(a,b):
    options = [a,b]
    a_b = df[df['Home Team'].isin(options)]
    a_b = a_b[a_b['Away Team'].isin(options)]
    a_b = pd.DataFrame(a_b)
    a_b.loc[((a_b['Home Team'] == a) & (a_b['FTR'] == 'H')) | ((a_b['Away Team'] == a) & (a_b['FTR'] == 'A')), 'Win_Value_a'] = 1
    a_b.loc[((a_b['Home Team'] == a) & (a_b['FTR'] == 'A')) | ((a_b['Away Team'] == a) & (a_b['FTR'] == 'H')), 'Win_Value_a'] = 0
    a_b.loc[((a_b['Home Team'] == b) & (a_b['FTR'] == 'H')) | ((a_b['Away Team'] == b) & (a_b['FTR'] == 'A')), 'Win_Value_b'] = 1
    a_b.loc[((a_b['Home Team'] == b) & (a_b['FTR'] == 'A')) | ((a_b['Away Team'] == b) & (a_b['FTR'] == 'H')), 'Win_Value_b'] = 0
    a_b = a_b.fillna(0.5) 

    # Home Team(a) value is 1 and away team(b) is 0
    a_b.loc[a_b['Home Team'] == a,'Home Team'] = 1 
    a_b.loc[a_b['Away Team'] == a,'Away Team'] = 1
    a_b.loc[a_b['Home Team'] == b,'Home Team'] = 0
    a_b.loc[a_b['Away Team'] == b,'Away Team'] = 0
    return a_b

# Gives us the dataframe of team_vs_all for all past years
def team_vs_all(team):
    options = [team]
    team_all = df[df['Home Team'].isin(options) | df['Away Team'].isin(options)]

    #If team a wins then Win_Value = 1, if loses then 0, if draw then 0.5  
    team_all = pd.DataFrame(team_all)
    team_all.loc[((team_all['Home Team'] == team) & (team_all['FTR'] == 'H')) | ((team_all['Away Team'] == team) & (team_all['FTR'] == 'A')), 'Win_Value'] = 1
    team_all.loc[((team_all['Home Team'] == team) & (team_all['FTR'] == 'A')) | ((team_all['Away Team'] == team) & (team_all['FTR'] == 'H')), 'Win_Value'] = 0
    team_all = team_all.fillna(0.5)
    return team_all

# Gives us the dataframe of all the home matches of a team
def home_matches(team):
    options = [team]
    home_all = df[df['Home Team'].isin(options)]
    home_all = pd.DataFrame(home_all)
    home_all.loc[((home_all['Home Team'] == team) & (home_all['FTR'] == 'H')), 'Home_Win_Value'] = 1
    home_all.loc[((home_all['Home Team'] == team) & (home_all['FTR'] == 'A')), 'Home_Win_Value'] = 0
    home_all = home_all.fillna(0.5) 
    return home_all

# Gives us the dataframe of all the away matches of a team
def away_matches(team):
    options = [team]
    away_all = df[df['Away Team'].isin(options)]
    away_all = pd.DataFrame(away_all)
    away_all.loc[((away_all['Away Team'] == team) & (away_all['FTR'] == 'H')), 'Away_Win_Value'] = 0
    away_all.loc[((away_all['Away Team'] == team) & (away_all['FTR'] == 'A')), 'Away_Win_Value'] = 1
    away_all = away_all.fillna(0.5) 
    return away_all

# Multi class classification (Win, Lose, Draw) 
def multiclassification(X,y,x,a,b):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    knn = KNeighborsClassifier(n_neighbors = 5)
    knn.fit(X_train, y_train)
    ans1 = knn.score(X_test,y_test)
    
    clf = LinearSVC().fit(X_train, y_train)
    ans2 = clf.score(X_test,y_test)

    X, y = shuffle(X, y, random_state=0)
    if ans1>ans2:
        knn.fit(X, y)
        print("Test accuracy :", ans1)
        ans = knn.predict(x)
    else:
        clf = LinearSVC().fit(X, y)
        print("Test accuracy :", ans2)
        ans = clf.predict(x)
    
    if ans == 0 :
        print(b,"Wins")
    if ans == 1 :
        print(a,"Wins")
    if ans == 2 :
        print("Draw")

# Regression of Home Win Probability
def regression_home(X,y,x,a):
    X_train, X_test, y_train, y_test1 = train_test_split(X, y, test_size=0.2, random_state = 0)
    knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X_train, y_train)
    error1 = np.sum(abs(knnreg.predict(X_test) - y_test1))

    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test2 = train_test_split(X_poly, y, test_size=0.2, random_state = 0)
    linreg = Ridge().fit(X_train, y_train)
    error2 = np.sum(abs(linreg.predict(X_test) - y_test2))
    
    X, y = shuffle(X, y, random_state=0)
    if error1['HP'] > error2['HP']:
        linreg = Ridge().fit(X, y)
        ans = linreg.predict(x)
    else:
        knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X, y)
        ans = knnreg.predict(x)
    return float(ans)
    
# Regression of Away Win Probability
def regression_away(X,y,x,b):
    X_train, X_test, y_train, y_test1 = train_test_split(X, y, test_size=0.2, random_state = 0)
    knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X_train, y_train)
    error1 = np.sum(abs(knnreg.predict(X_test) - y_test1))

    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test2 = train_test_split(X_poly, y, test_size=0.2, random_state = 0)
    linreg = Ridge().fit(X_train, y_train)
    error2 = np.sum(abs(linreg.predict(X_test) - y_test2))
    
    X, y = shuffle(X, y, random_state=0)
    if error1['AP'] > error2['AP']:
        linreg = Ridge().fit(X, y)
        ans = linreg.predict(x)
    else:
        knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X, y)
        ans = knnreg.predict(x)
    return float(ans)

# Regression of Draw Probability
def regression_draw(X,y,x):
    X_train, X_test, y_train, y_test1 = train_test_split(X, y, test_size=0.2, random_state = 0)
    knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X_train, y_train)
    error1 = np.sum(abs(knnreg.predict(X_test) - y_test1))

    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test2 = train_test_split(X_poly, y, test_size=0.2, random_state = 0)
    linreg = Ridge().fit(X_train, y_train)
    error2 = np.sum(abs(linreg.predict(X_test) - y_test2))
    
    X, y = shuffle(X, y, random_state=0)
    if error1['DP'] > error2['DP']:
        linreg = Ridge().fit(X, y)
        ans = linreg.predict(x)
    else:
        knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X, y)
        ans = knnreg.predict(x)
    return float(ans)

Enter the name of teams 

In [269]:
a = input("Enter Home Team :") #Home Team
b = input("Enter Away Team :") #Away Team

Enter Home Team :Chelsea
Enter Away Team :Arsenal


Processing of data to get the required features for Prediction of Win Probability

In [270]:
a_all = team_vs_all(a) # team a vs all other teams data
b_all = team_vs_all(b) # team b vs all other team data
a_b = a_vs_b(a,b) # team a vs team b data
a_home = home_matches(a) # team a all home matches
b_home = home_matches(b) # team b all home matches
a_away = away_matches(a) # team a all away matches
b_away = away_matches(b) # team b all away matches

# It takes average of last 6 win valus of team a in the matches team a vs team b
values = []
list_values = []
sum = 0

for value in a_b['Win_Value_a']:
    values.append(value)

for i in range(len(values)):
    if i<6:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-6]
        prob = sum / 6
    list_values.append(prob) 

a_b["Win_last6_a"] = list_values

# It takes average of last 6 win valus of team b in the matches team a vs team b
values = []
list_values = []
sum = 0

for value in a_b['Win_Value_b']:
    values.append(value)

for i in range(len(values)):
    if i<6:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-6]
        prob = sum / 6
    list_values.append(prob) 

a_b["Win_last6_b"] = list_values

# Similar as above but average of last 5 home win value of team a 
values = []
list_values = []
sum = 0

for value in a_home['Home_Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_home["Home_Win_Prob"] = list_values
a_home['Home_Win_Prob'] = np.roll(a_home['Home_Win_Prob'], shift=1)

# Similar as above but average of last 5 home win value of team b
values = []
list_values = []
sum = 0

for value in b_home['Home_Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_home["Home_Win_Prob"] = list_values
b_home['Home_Win_Prob'] = np.roll(b_home['Home_Win_Prob'], shift=1)

# Similar as above but average of last 5 away win value of team a
values = []
list_values = []
sum = 0

for value in a_away['Away_Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_away["Away_Win_Prob"] = list_values
a_away['Away_Win_Prob'] = np.roll(a_away['Away_Win_Prob'], shift=1)

# Similar as above but average of last 5 away win value of team b
values = []
list_values = []
sum = 0

for value in b_away['Away_Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_away["Away_Win_Prob"] = list_values
b_away['Away_Win_Prob'] = np.roll(b_away['Away_Win_Prob'], shift=1)

# Similar as above but average of last 10 win value of team a vs all teams
values = []
list_values = []
sum = 0

for value in a_all['Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<10:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-10]
        prob = sum / 10
    list_values.append(prob) 

a_all["Win_Prob_Last10_a"] = list_values
a_all['Win_Prob_Last10_a'] = np.roll(a_all['Win_Prob_Last10_a'], shift=1)

values = []
list_values = []
sum = 0

for value in a_all['Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["Win_Prob_Last5_a"] = list_values
a_all['Win_Prob_Last5_a'] = np.roll(a_all['Win_Prob_Last5_a'], shift=1)

# Similar as above but average of last 10 win value of team b vs all teams
values = []
list_values = []
sum = 0

for value in b_all['Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<10:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-10]
        prob = sum / 10
    list_values.append(prob) 

b_all["Win_Prob_Last10_b"] = list_values
b_all['Win_Prob_Last10_b'] = np.roll(b_all['Win_Prob_Last10_b'], shift=1)

values = []
list_values = []
sum = 0

for value in b_all['Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["Win_Prob_Last5_b"] = list_values
b_all['Win_Prob_Last5_b'] = np.roll(b_all['Win_Prob_Last5_b'], shift=1)

# Take average of Home team Shots/Away Team shots average on target for both teams
values = []
list_values = []
sum = 0

for value in b_all['HSTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["HSTP_avg"] = list_values
b_all['HSTP_avg'] = np.roll(b_all['HSTP_avg'], shift=1)

values = []
list_values = []
sum = 0

for value in b_all['ASTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["ASTP_avg"] = list_values
b_all['ASTP_avg'] = np.roll(b_all['ASTP_avg'], shift=1)

values = []
list_values = []
sum = 0

for value in a_all['ASTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["ASTP_avg"] = list_values
a_all['ASTP_avg'] = np.roll(a_all['ASTP_avg'], shift=1)

values = []
list_values = []
sum = 0

for value in a_all['HSTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["HSTP_avg"] = list_values
a_all['HSTP_avg'] = np.roll(a_all['HSTP_avg'], shift=1)


Merging the features with the Dataset

In [276]:
# Merging data set
data = pd.merge(a_b,a_home[['Match_Id','Home_Win_Prob']], on='Match_Id',how = 'left')
b_home = b_home[['Match_Id','Home_Win_Prob']]
data.loc[data.Match_Id.isin(b_home.Match_Id), ['Home_Win_Prob']] = b_home.loc[b_home.Match_Id.isin(data.Match_Id),['Home_Win_Prob']].values

data = pd.merge(data,a_away[['Match_Id','Away_Win_Prob']], on='Match_Id',how = 'left')
b_away = b_away[['Match_Id','Away_Win_Prob']]
data.loc[data.Match_Id.isin(b_away.Match_Id), ['Away_Win_Prob']] = b_away.loc[b_away.Match_Id.isin(data.Match_Id),['Away_Win_Prob']].values

data = pd.merge(data,a_all[['Match_Id','ASTP_avg']], on='Match_Id',how = 'left')
data.loc[data.Match_Id.isin(b_all.Match_Id), ['ASTP_avg']] = b_all.loc[b_all.Match_Id.isin(data.Match_Id),['ASTP_avg']].values
data = pd.merge(data,a_all[['Match_Id','HSTP_avg']], on='Match_Id',how = 'left')
data.loc[data.Match_Id.isin(b_all.Match_Id), ['HSTP_avg']] = b_all.loc[b_all.Match_Id.isin(data.Match_Id),['HSTP_avg']].values

data = pd.merge(data,a_all[['Match_Id','Win_Prob_Last10_a']], on='Match_Id', how='left')
data = pd.merge(data,b_all[['Match_Id','Win_Prob_Last10_b']], on='Match_Id', how='left')
data = pd.merge(data,a_all[['Match_Id','Win_Prob_Last5_a']], on='Match_Id', how='left')
data = pd.merge(data,b_all[['Match_Id','Win_Prob_Last5_b']], on='Match_Id', how='left')

data['Win_last6_a'] = np.roll(data['Win_last6_a'], shift=1)
data['Win_last6_b'] = np.roll(data['Win_last6_b'], shift=1)

columns = ['Home Team','Away Team', 'Win_last6_a', 'Win_last6_b', 'Home_Win_Prob', 'Away_Win_Prob', 'Win_Prob_Last10_a', 'Win_Prob_Last10_b', 'Win_Prob_Last5_a', 'Win_Prob_Last5_b']
x = data[columns]
x = x[0:1]
x['Home Team'] = 1
x['Away Team'] = 0
x['Home_Win_Prob'] = a_home.iloc[0]['Home_Win_Prob']
x['Away_Win_Prob'] = b_away.iloc[0]['Away_Win_Prob']
x['Win_Prob_Last10_a'] = a_all.iloc[0]['Win_Prob_Last10_a']
x['Win_Prob_Last5_a'] = a_all.iloc[0]['Win_Prob_Last5_a']
x['Win_Prob_Last10_b'] = b_all.iloc[0]['Win_Prob_Last10_b']
x['Win_Prob_Last5_b'] = b_all.iloc[0]['Win_Prob_Last5_b']

X = data[columns]
X = X[2:]

data.loc[((data['Home Team'] == 0) & (data['FTR'] == 'H')) | ((data['Away Team'] == 0) & (data['FTR'] == 'A')), 'Win Team'] = 0
data.loc[((data['Home Team'] == 1) & (data['FTR'] == 'H')) | ((data['Away Team'] == 1) & (data['FTR'] == 'A')), 'Win Team'] = 1
data.loc[(data['FTR'] == 'D'), 'Win Team'] = 2 # Labeled value of draw matches for y column

y = data[['Win Team']]
y = y[2:]
y = y.values.ravel()

#####MultiClass Classification (3 classes) 
######X = Input features dataset
######y = Labeled output values
######x = The value of features on which we want outcome of prediction algorithm
######a = Home Team
######b = Away Team

In [277]:
multiclassification(X,y,x,a,b)

Test accuracy : 0.5714285714285714
Chelsea Wins


Data Manipulation for Win probability prediction

In [273]:
columns = ['Home Team','Away Team','ASTP_avg','HSTP_avg', 'Win_last6_a', 'Win_last6_b', 'Home_Win_Prob', 'Away_Win_Prob', 'Win_Prob_Last10_a', 'Win_Prob_Last10_b', 'Win_Prob_Last5_a', 'Win_Prob_Last5_b']
X = data[columns]
X = X[2:]

y_hp = data[['HP']]
y_hp = y_hp[2:]
y_ap = data[['AP']]
y_ap = y_ap[2:]
y_dp = data[['DP']]
y_dp = y_dp[2:]

x = data[columns]
x = x[0:1]
x['Home Team'] = 1
x['ASTP_avg'] = b_all.iloc[0]['ASTP_avg']
x['HSTP_avg'] = a_all.iloc[0]['HSTP_avg']
x['Home_Win_Prob'] = a_home.iloc[0]['Home_Win_Prob']
x['Away_Win_Prob'] = b_away.iloc[0]['Away_Win_Prob']
x['Win_Prob_Last10_a'] = a_all.iloc[0]['Win_Prob_Last10_a']
x['Win_Prob_Last5_a'] = a_all.iloc[0]['Win_Prob_Last5_a']
x['Win_Prob_Last10_b'] = b_all.iloc[0]['Win_Prob_Last10_b']
x['Win_Prob_Last5_b'] = b_all.iloc[0]['Win_Prob_Last5_b']

#####Below three function provide probability of upcoming a vs b match
#####It provides win probability of Team a and Team b and Draw probabilty of the match
#####y_hp = Home Win probability from Bet365 data
#####y_ap = Away Win probability from Bet365 data
#####y_dp = Draw probability from Bet365 data
#####X = Input features dataset
######x = The value of features on which we want outcome of prediction algorithm
#####a = Home Team
#####b = Away Team

In [274]:
home = regression_home(X,y_hp,x,a)
print(a,"Win Probability is", home)

away = regression_away(X,y_ap,x,b)
print(b,"Win Probability is", away)

draw = regression_draw(X,y_dp,x)
print("Draw Probability is", draw)

if ((draw > away) and (draw > home)):
    print("Draw")
elif (home > away):
    print(a, "Wins")
else :
    print(b, "Wins")

Chelsea Win Probability is 0.45456485743782693
Arsenal Win Probability is 0.272447449051066
Draw Probability is 0.27298769351110697
Chelsea Wins
